In [1]:
from sqlalchemy import create_engine, text
from bs4 import BeautifulSoup as bs
import datetime as dt
import pandas as pd
import numpy as np
import warnings
import requests
import pymssql
import locale
import glob
import os

warnings.filterwarnings('ignore')
pd.set_option('use_inf_as_na', True)
pd.options.display.max_columns = None
pd.set_option('float_format', '{:.2f}'.format)
locale.setlocale(locale.LC_ALL, 'ru_RU.UTF-8')

'ru_RU.UTF-8'

In [ ]:
conn = pymssql.connect(server='AX-SQL', database='Staging')
cursor = conn.cursor()

sql = f'''
SELECT [Код склада], [Название склада], [Название товара], [Артикул поставщика], [Остаток], 
[Проба], [Чистый вес], [Товарное направление], [Товарная группа], [Тип изделия 1], [Тип изделия 2],
[Тип изделия 3], [Группа цен], [Скидка на изделии], [Запрет скидки], [Дизайн], [Бренд], [Дата закупки], 
[Цена Розн., за шт], [Гендерный признак], [Ценовая корзина], [Товарный кластер], [Внутренний артикул],
[ID Сайта], [Цена Закупки за 1 шт. (упр сс с НДС за единицу)]
from [Staging].[Reports].[Remainings_8h]
where [Товарное направление] in (N'БК', N'ЦБ', N'СИ', N'ИФ', N'ПДК', N'ДК', N'ВЫРИЦА', N'ЧАСЫ')
'''

sql_query = pd.read_sql_query(sql, conn)
stock = pd.DataFrame(sql_query)

In [2]:
conn = pymssql.connect(server='AX-SQL', database='Staging')
cursor = conn.cursor()

sql = f'''
SELECT [Код склада], [Название склада], [Название товара], [Артикул поставщика], [Остаток], 
[Проба], [Чистый вес], [Товарное направление], [Товарная группа], [Тип изделия 1], [Тип изделия 2],
[Тип изделия 3], [Группа цен], [Скидка на изделии], [Запрет скидки], [Дизайн], [Бренд], [Дата закупки], 
[Цена Розн., за шт], [Гендерный признак], [Ценовая корзина], [Товарный кластер], [Внутренний артикул],
[ID Сайта], [Цена Закупки за 1 шт. (упр сс с НДС за единицу)]
from [Staging].[Reports].[Remainings_8h]
where [Товарное направление] in (N'БК', N'ЦБ', N'СИ', N'ИФ', N'ПДК', N'ДК', N'ВЫРИЦА', N'ЧАСЫ')
'''

sql_query = pd.read_sql_query(sql, conn)
stock = pd.DataFrame(sql_query)

In [3]:
df = stock

In [4]:
ids = df.groupby(by = 'Артикул поставщика', as_index=False)['Остаток'].sum()

In [5]:
ids = ids.query('Остаток >= 50')

In [6]:
df.columns

Index(['Код склада', 'Название склада', 'Название товара',
       'Артикул поставщика', 'Остаток', 'Проба', 'Чистый вес',
       'Товарное направление', 'Товарная группа', 'Тип изделия 1',
       'Тип изделия 2', 'Тип изделия 3', 'Группа цен', 'Скидка на изделии',
       'Запрет скидки', 'Дизайн', 'Бренд', 'Дата закупки', 'Цена Розн., за шт',
       'Гендерный признак', 'Ценовая корзина', 'Товарный кластер',
       'Внутренний артикул', 'ID Сайта',
       'Цена Закупки за 1 шт. (упр сс с НДС за единицу)'],
      dtype='object')

In [7]:
idss = df[['Код склада', 'Название склада', 'Название товара',
    'Артикул поставщика', 'Остаток', 'Проба', 'Чистый вес',
    'Товарное направление', 'Товарная группа', 'Тип изделия 1',
    'Тип изделия 2', 'Тип изделия 3', 'Группа цен', 'Скидка на изделии',
    'Запрет скидки', 'Дизайн', 'Бренд', 'Дата закупки', 'Цена Розн., за шт',
    'Гендерный признак', 'Ценовая корзина', 'Товарный кластер',
    'Внутренний артикул', 'ID Сайта',
    'Цена Закупки за 1 шт. (упр сс с НДС за единицу)']]

In [8]:
# stock.to_csv(r'C:\Users\Trenkin.Sergey\Desktop\sales\\выгрузка.csv', index=False)

In [9]:
ids = idss.merge(ids, how='left', on='Артикул поставщика')

In [10]:
ids = ids.drop_duplicates(subset=['Артикул поставщика'])

In [12]:
ids = ids.query('Остаток_y >= 50')

In [13]:
ids = ids[~ids['Товарное направление'].str.contains('ФУТЛЯРЫ|БУ')]
ids = ids[ids['Проба'].str.contains('585|925')]

In [14]:
ids['Товарное направление'].unique().tolist()

['ЦБ', 'ИФ', 'ДК', 'СИ', 'БК', 'ПДК', 'ВЫРИЦА', 'ЧАСЫ']

In [15]:
ids['Проба'].value_counts()

Проба
585    11621
925     8677
Name: count, dtype: int64

In [16]:
stock['Товарное направление'].value_counts(dropna=False)

Товарное направление
СИ        2767247
ИФ         905050
БК         839826
ЦБ         647023
ПДК        601238
ДК         569084
ВЫРИЦА      85064
ЧАСЫ        76767
Name: count, dtype: int64

In [17]:
sales_may1 = pd.read_excel(r'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Выгрузки из DWH\Продажи по КБК из DWH new\2024-05-01_2024-05-09.xlsx')
sales_may2 = pd.read_excel(r'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Выгрузки из DWH\Продажи по КБК из DWH new\2024-05-10_2024-05-19.xlsx')
sales_may3 = pd.read_excel(r'C:\Users\Trenkin.Sergey\Desktop\Книга13.xlsx')

In [18]:
sales_may3['Артикул поставщика'] = sales_may3['Артикул поставщика'].astype(str)

In [19]:
sales_may = pd.concat([sales_may1, sales_may2, sales_may3], ignore_index=True)

In [20]:
articles = ids['Артикул поставщика'].tolist()
arts = ids['ID Сайта'].tolist()

In [21]:
sales_may = sales_may.dropna(subset=['Количество', 'Артикул поставщика'])

In [22]:
sales_may['Количество'].value_counts(dropna=False)

Количество
1.00     724985
-1.00      3503
Name: count, dtype: int64

In [23]:
sales_may = sales_may.groupby(by = 'Артикул поставщика', as_index=False)[['Количество']].sum()

In [24]:
sales_may.columns = ['article', 'count']

In [25]:
sales_may = sales_may.query('article in @articles')

In [26]:
sales_may.columns = ['Артикул поставщика', 'Количество']
sales_may

,Артикул поставщика,Количество
6,0-017,56.00
11,00-03-0858,7.00
13,00008,9.00
16,000094-0#ХХ,3.00
19,00016Ч,7.00
...,...,...
41708,с300-1156НЕФ,10.00
41709,с35-04,37.00
41710,с462-20,2.00
41711,с9-16,35.00


In [27]:
ids = ids.merge(sales_may, how='left', on='Артикул поставщика')

In [28]:
ids['Количество'].value_counts()

Количество
1.00      2995
2.00      2421
3.00      1734
4.00      1378
5.00       992
          ... 
333.00       1
171.00       1
423.00       1
289.00       1
558.00       1
Name: count, Length: 331, dtype: int64

In [29]:
ids.to_excel('result.xlsx', index=False)

In [30]:
images = pd.read_excel(r'C:\Users\Trenkin.Sergey\Desktop\Артикул-фото 2024.06.11_.xlsx')

In [31]:
images = images.drop_duplicates(subset = ['article'])

In [32]:
images['article'] = images['article'].astype(str)

In [33]:
images = images.query('article in @arts')

In [34]:
images

,article,sort,published,Ссылка на изображение,Ссылка на товар
2,1000091,3,True,https://static2.585.cloud/media/products/f2d30...,https://www.585zolotoy.ru/catalog/products/100...
5,1000181,1,True,https://static2.585.cloud/media/products/38b3b...,https://www.585zolotoy.ru/catalog/products/100...
6,1000188,3,True,https://static2.585.cloud/media/products/f3bf4...,https://www.585zolotoy.ru/catalog/products/100...
16,1000339,1,True,https://static2.585.cloud/media/products/b141d...,https://www.585zolotoy.ru/catalog/products/100...
19,1000477,1,True,https://static2.585.cloud/media/products/b1000...,https://www.585zolotoy.ru/catalog/products/100...
...,...,...,...,...,...
181631,8380093,4,True,https://static2.585.cloud/media/products/99ab5...,https://www.585zolotoy.ru/catalog/products/838...
181637,8380097,5,True,https://static2.585.cloud/media/products/a3245...,https://www.585zolotoy.ru/catalog/products/838...
181642,8380099,4,True,https://static2.585.cloud/media/products/ffa1a...,https://www.585zolotoy.ru/catalog/products/838...
181646,8380101,4,True,https://static2.585.cloud/media/products/1c202...,https://www.585zolotoy.ru/catalog/products/838...


In [35]:
images = images[['article', 'Ссылка на изображение', 'published']]

In [36]:
ids

,Код склада,Название склада,Название товара,Артикул поставщика,Остаток_x,Проба,Чистый вес,Товарное направление,Товарная группа,Тип изделия 1,Тип изделия 2,Тип изделия 3,Группа цен,Скидка на изделии,Запрет скидки,Дизайн,Бренд,Дата закупки,"Цена Розн., за шт",Гендерный признак,Ценовая корзина,Товарный кластер,Внутренний артикул,ID Сайта,Цена Закупки за 1 шт. (упр сс с НДС за единицу),Остаток_y,Количество
0,4616,"Мытищи, Мира, 51, ТРЦ «ИЮНЬ»",Браслет Веревка пуст,431-00-0060-30333,1.00,585,2.09,ЦБ,ЦБ БРАСЛЕТЫ,БРАСЛЕТ,БЕЗ КАМНЕЙ,КРАСНЫЙ,ЦБ_ЮИТ_ПУСТОТЕЛЫЕ_КОРДА <3 ГР,25.00,Нет,ВЕРЕВКА ПУСТ,Значение по умолчанию,2023-02-15,34970.00,женские,ДЕШЕВАЯ,7645_ЦБ,431-00-0060-30333_ЦБ,3778045,4782.59,3501.00,140.00
1,4616,"Мытищи, Мира, 51, ТРЦ «ИЮНЬ»",Браслет Ромб двойной пуст с а/г,411-01-0035-30176,1.00,585,0.65,ЦБ,ЦБ БРАСЛЕТЫ,БРАСЛЕТ,БЕЗ КАМНЕЙ,КРАСНЫЙ,"ЦБ_ПУСТОТЕЛЫЕ <=1,5 ГРАММ",30.00,Нет,РОМБ ДВОЙНОЙ ПУСТ,ADRIATICA,2023-04-12,12513.00,женские,ДЕШЕВАЯ,7730_ЦБ,411-01-0035-30176_ЦБ,1674395,1436.74,2635.00,147.00
2,4616,"Мытищи, Мира, 51, ТРЦ «ИЮНЬ»","Серьги, фианит",1241689,1.00,585,1.98,ИФ,ИФ СЕРЬГИ,СЕРЬГИ,С ФИАНИТАМИ,КРАСНЫЙ,ИФ.БК_СР,34.00,Нет,ИФ С ОДНИМ КАМНЕМ,Значение по умолчанию,2023-11-30,39382.00,женские,СРЕДНЯЯ,ИФ_1044,1241689_ИФ,2735473,6802.13,506.00,25.00
3,4616,"Мытищи, Мира, 51, ТРЦ «ИЮНЬ»",серьги с бриллиантом с родир.,2436-151-00-00,1.00,585,3.06,ДК,ДК СЕРЬГИ,СЕРЬГИ,С ДРАГОЦЕННЫМИ КАМНЯМИ,КРАСНЫЙ,ДК_СРЕДНЯЯ,66.00,Нет,ДК С ОДНИМ КАМНЕМ,Значение по умолчанию,None,163994.00,женские,СРЕДНЯЯ,ДК_2090,2436-151-00-00_ДК,4058155,10755.11,334.00,7.00
4,4616,"Мытищи, Мира, 51, ТРЦ «ИЮНЬ»",Подвес род.,2035076/9,1.00,925,0.50,СИ,СИ ПОДВЕС БК,ПОДВЕС ДЕКОРАТИВНЫЙ,БЕЗ КАМНЕЙ,БЕЛЫЙ С РОДИРОВАНИЕМ,СИ_ПОДВЕС_ДЕКОР_СРЕДН,40.00,Нет,СИ ПРОЧЕЕ,Значение по умолчанию,2024-02-16,1150.00,женские,ЛЕГКОВЕС,6400_СИ,2035076/9_СИ,3558970,195.00,1088.00,131.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20293,3075,Центральный склад Регент Голд,"Кольцо с бриллиантами выращ., с родир.",20-12-Ф000-13341,1.00,585,1.09,ДК,ДК КОЛЬЦА,КОЛЬЦО,С ИСКУССТВЕННЫМИ КАМНЯМИ,БЕЛЫЙ,ДК_ДОРОГАЯ,0.00,Нет,ДК МНОГОКАМЕНКА,Значение по умолчанию,2024-06-11,NaN,женские,ДОРОГАЯ,ДК_2027,20-12-Ф000-13341 _ДК,,24924.21,95.00,NaN
20294,3075,Центральный склад Регент Голд,Подвес икона Семистрельная,52-0Z-0000-13522,1.00,585,1.19,БК,БК ПОДВЕС КУЛЬТ,ПОДВЕС КУЛЬТОВЫЙ,БЕЗ КАМНЕЙ,КРАСНЫЙ,ИФ.БК_ДЕШ,0.00,Нет,ИФ ИКОНА,Значение по умолчанию,2024-06-08,NaN,унисекс,ДЕШЕВАЯ,БК_1568,52-0Z-0000-13522_БК,,4197.12,50.00,NaN
20295,3075,Центральный склад Регент Голд,Подвес,50-00-0000-13608#ER,1.00,585,1.09,БК,БК ПОДВЕС ДЕКОР,ПОДВЕС ДЕКОРАТИВНЫЙ,БЕЗ КАМНЕЙ,КРАСНЫЙ,ИФ.БК_СР,0.00,Нет,ИФ ПРОЧЕЕ,Значение по умолчанию,2024-06-08,NaN,Унисекс,СРЕДНЯЯ,БК_1593,50-00-0000-13608#ER_БК,,3980.90,55.00,NaN
20296,3075,Центральный склад Регент Голд,"Подвес позолота, культивированный жемчуг",18.104,1.00,925,2.30,ВЫРИЦА,ВЫРИЦА СЕРЕБРО,ПОДВЕС,С ЖЕМЧУГОМ,ЖЕЛТЫЙ,СИ_ВЫРИЦА_ДЕШ,0.00,Нет,В_ПРОЧЕЕ,ВЫРИЦА,2024-06-11,NaN,Значение по умолчанию,СРЕДНЯЯ,,18.104_ВЫРИЦА,,1912.00,101.00,NaN


In [37]:
images.columns = ['ID Сайта', 'Ссылка на изображение', 'published']

In [38]:
ids = ids.merge(images, how='left', on='ID Сайта')

In [39]:
ids.columns

Index(['Код склада', 'Название склада', 'Название товара',
       'Артикул поставщика', 'Остаток_x', 'Проба', 'Чистый вес',
       'Товарное направление', 'Товарная группа', 'Тип изделия 1',
       'Тип изделия 2', 'Тип изделия 3', 'Группа цен', 'Скидка на изделии',
       'Запрет скидки', 'Дизайн', 'Бренд', 'Дата закупки', 'Цена Розн., за шт',
       'Гендерный признак', 'Ценовая корзина', 'Товарный кластер',
       'Внутренний артикул', 'ID Сайта',
       'Цена Закупки за 1 шт. (упр сс с НДС за единицу)', 'Остаток_y',
       'Количество', 'Ссылка на изображение', 'published'],
      dtype='object')

In [40]:
ids.columns = ['Код склада', 'Название склада', 'Название товара',
    'Артикул поставщика', 'Остаток_x', 'Проба', 'Чистый вес',
    'Товарное направление', 'Товарная группа', 'Тип изделия 1',
    'Тип изделия 2', 'Тип изделия 3', 'Группа цен', 'Скидка на изделии',
    'Запрет скидки', 'Дизайн', 'Бренд', 'Дата закупки', 'Цена Розн., за шт',
    'Гендерный признак', 'Ценовая корзина', 'Товарный кластер',
    'Внутренний артикул', 'ID Сайта',
    'Цена Закупки за 1 шт. (упр сс с НДС за единицу)', 'Остаток_y',
    'Количество продаж за май', 'Ссылка на изображение', 'published']

In [41]:
ids['Товарная группа'].unique().tolist()

['ЦБ БРАСЛЕТЫ',
 'ИФ СЕРЬГИ',
 'ДК СЕРЬГИ',
 'СИ ПОДВЕС БК',
 'СИ СЕРЬГИ\xa0 ЦВ/К',
 'СИ ЖЕМЧУГ',
 'ИФ КОЛЬЦА',
 'СИ ПОДВЕС\xa0ЦВ/К',
 'СИ БРАСЛЕТ БК',
 'БК КОЛЬЦА ОБРУЧ',
 'СИ СЕРЬГИ РАЗН',
 'СИ КОЛЬЦО ЦВ/К',
 'СИ КОЛЬЦО БК',
 'ПДК СЕРЬГИ',
 'СИ ЦЕПЬ',
 'СИ КЕРАМИКА',
 'ИФ ПОДВЕС ДЕКОР',
 'СИ ПОДВЕС ИФ',
 'БК ПОДВЕС КУЛЬТ',
 'ДК ПОДВЕСКИ',
 'ЦБ ЦЕПИ',
 'ПДК КОЛЬЦА',
 'СИ ШАРМЫ',
 'СИ КОЛЬЦО ИФ',
 'ДК КОЛЬЦА ОБРУЧ',
 'БК ПОДВЕС ДЕКОР',
 'СИ ОБРУЧАЛЬНОЕ',
 'СИ СЕРЬГИ БК',
 'СИ КОЛЬЕ ЦВ/К',
 'СИ КОЛЬЕ СК',
 'СИ БРАСЛЕТ СК',
 'БК КОЛЬЦА',
 'ДК КОЛЬЦА',
 'ИФ ПОДВЕС КУЛЬТ',
 'СИ АКСЕССУАР',
 'ПДК ПОДВЕСКИ',
 'СИ ПОДАРКИ КРОСС',
 'СИ СЕРЬГИ ИФ',
 'БК СЕРЬГИ',
 'ИФ КОЛЬЦА ОБРУЧ',
 'СИ КОЛЬЕ С ЭМАЛЬЮ',
 'ОПТ БК',
 'БК ПЕЧАТКИ',
 'ПДК ПЕЧАТКИ',
 'СИ КОЛЬЦО ПЕЧАТ',
 'СИ КОЛЬЕ БК',
 'ДК ПЕЧАТКИ',
 'ИФ ПЕЧАТКИ',
 'СИ БРАСЛЕТ ЦВ/К',
 'ВЫРИЦА СЕРЕБРО',
 'ОПТ ИФ',
 'ЧАСЫ',
 'СИ БРАСЛЕТ БК Декор',
 'СИ ЭМАЛЬ',
 'ОПТ ПДК',
 'СИ ЯНТАРЬ',
 'СИ ПОСУДА',
 'ВЫРИЦА ЗОЛОТО',
 'ОПТ КОНГО',
 'ОПТ ОБРУЧИ',
 'БК 

In [42]:
ids = ids[~ids['Товарная группа'].str.contains('АКСЕССУАР|ОПТ|ПОСУДА|КРОСС|ЦВ/К|РАЗН|КЕРАМИКА|ЭМАЛЬ|ЯНТАРЬ')]

In [43]:
ids = ids.drop(columns=['Остаток_x'])

In [49]:
ids

,Код склада,Название склада,Название товара,Артикул поставщика,Проба,Чистый вес,Товарное направление,Товарная группа,Тип изделия 1,Тип изделия 2,Тип изделия 3,Группа цен,Скидка на изделии,Запрет скидки,Дизайн,Бренд,Дата закупки,"Цена Розн., за шт",Гендерный признак,Ценовая корзина,Товарный кластер,Внутренний артикул,ID Сайта,Цена Закупки за 1 шт. (упр сс с НДС за единицу),Остаток_y,Количество продаж за май,Ссылка на изображение,published,price_personal
0,4616,"Мытищи, Мира, 51, ТРЦ «ИЮНЬ»",Браслет Веревка пуст,431-00-0060-30333,585,2.09,ЦБ,ЦБ БРАСЛЕТЫ,БРАСЛЕТ,БЕЗ КАМНЕЙ,КРАСНЫЙ,ЦБ_ЮИТ_ПУСТОТЕЛЫЕ_КОРДА <3 ГР,25.00,Нет,ВЕРЕВКА ПУСТ,Значение по умолчанию,2023-02-15,34970.00,женские,ДЕШЕВАЯ,7645_ЦБ,431-00-0060-30333_ЦБ,3778045,4782.59,3501.00,140.00,https://static2.585.cloud/media/products/d5758...,True,12589.20
1,4616,"Мытищи, Мира, 51, ТРЦ «ИЮНЬ»",Браслет Ромб двойной пуст с а/г,411-01-0035-30176,585,0.65,ЦБ,ЦБ БРАСЛЕТЫ,БРАСЛЕТ,БЕЗ КАМНЕЙ,КРАСНЫЙ,"ЦБ_ПУСТОТЕЛЫЕ <=1,5 ГРАММ",30.00,Нет,РОМБ ДВОЙНОЙ ПУСТ,ADRIATICA,2023-04-12,12513.00,женские,ДЕШЕВАЯ,7730_ЦБ,411-01-0035-30176_ЦБ,1674395,1436.74,2635.00,147.00,https://static2.585.cloud/media/products/6bedb...,True,4204.37
2,4616,"Мытищи, Мира, 51, ТРЦ «ИЮНЬ»","Серьги, фианит",1241689,585,1.98,ИФ,ИФ СЕРЬГИ,СЕРЬГИ,С ФИАНИТАМИ,КРАСНЫЙ,ИФ.БК_СР,34.00,Нет,ИФ С ОДНИМ КАМНЕМ,Значение по умолчанию,2023-11-30,39382.00,женские,СРЕДНЯЯ,ИФ_1044,1241689_ИФ,2735473,6802.13,506.00,25.00,https://static2.585.cloud/media/products/ff494...,True,12476.22
3,4616,"Мытищи, Мира, 51, ТРЦ «ИЮНЬ»",серьги с бриллиантом с родир.,2436-151-00-00,585,3.06,ДК,ДК СЕРЬГИ,СЕРЬГИ,С ДРАГОЦЕННЫМИ КАМНЯМИ,КРАСНЫЙ,ДК_СРЕДНЯЯ,66.00,Нет,ДК С ОДНИМ КАМНЕМ,Значение по умолчанию,None,163994.00,женские,СРЕДНЯЯ,ДК_2090,2436-151-00-00_ДК,4058155,10755.11,334.00,7.00,https://static2.585.cloud/media/products/501d7...,True,26763.82
4,4616,"Мытищи, Мира, 51, ТРЦ «ИЮНЬ»",Подвес род.,2035076/9,925,0.50,СИ,СИ ПОДВЕС БК,ПОДВЕС ДЕКОРАТИВНЫЙ,БЕЗ КАМНЕЙ,БЕЛЫЙ С РОДИРОВАНИЕМ,СИ_ПОДВЕС_ДЕКОР_СРЕДН,40.00,Нет,СИ ПРОЧЕЕ,Значение по умолчанию,2024-02-16,1150.00,женские,ЛЕГКОВЕС,6400_СИ,2035076/9_СИ,3558970,195.00,1088.00,131.00,https://static2.585.cloud/media/products/21c90...,True,331.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20293,3075,Центральный склад Регент Голд,"Кольцо с бриллиантами выращ., с родир.",20-12-Ф000-13341,585,1.09,ДК,ДК КОЛЬЦА,КОЛЬЦО,С ИСКУССТВЕННЫМИ КАМНЯМИ,БЕЛЫЙ,ДК_ДОРОГАЯ,0.00,Нет,ДК МНОГОКАМЕНКА,Значение по умолчанию,2024-06-11,NaN,женские,ДОРОГАЯ,ДК_2027,20-12-Ф000-13341 _ДК,,24924.21,95.00,NaN,NaN,NaN,NaN
20294,3075,Центральный склад Регент Голд,Подвес икона Семистрельная,52-0Z-0000-13522,585,1.19,БК,БК ПОДВЕС КУЛЬТ,ПОДВЕС КУЛЬТОВЫЙ,БЕЗ КАМНЕЙ,КРАСНЫЙ,ИФ.БК_ДЕШ,0.00,Нет,ИФ ИКОНА,Значение по умолчанию,2024-06-08,NaN,унисекс,ДЕШЕВАЯ,БК_1568,52-0Z-0000-13522_БК,,4197.12,50.00,NaN,NaN,NaN,NaN
20295,3075,Центральный склад Регент Голд,Подвес,50-00-0000-13608#ER,585,1.09,БК,БК ПОДВЕС ДЕКОР,ПОДВЕС ДЕКОРАТИВНЫЙ,БЕЗ КАМНЕЙ,КРАСНЫЙ,ИФ.БК_СР,0.00,Нет,ИФ ПРОЧЕЕ,Значение по умолчанию,2024-06-08,NaN,Унисекс,СРЕДНЯЯ,БК_1593,50-00-0000-13608#ER_БК,,3980.90,55.00,NaN,NaN,NaN,NaN
20296,3075,Центральный склад Регент Голд,"Подвес позолота, культивированный жемчуг",18.104,925,2.30,ВЫРИЦА,ВЫРИЦА СЕРЕБРО,ПОДВЕС,С ЖЕМЧУГОМ,ЖЕЛТЫЙ,СИ_ВЫРИЦА_ДЕШ,0.00,Нет,В_ПРОЧЕЕ,ВЫРИЦА,2024-06-11,NaN,Значение по умолчанию,СРЕДНЯЯ,,18.104_ВЫРИЦА,,1912.00,101.00,NaN,NaN,NaN,NaN


In [45]:
ids['price_personal'] = np.where(
    ids['Запрет скидки'] == 'Да', ids['Цена Розн., за шт'], 
    np.where(
        ids['Проба'] == 925, ids['Цена Розн., за шт'] * 0.7 * 0.8 * (1 - (ids['Скидка на изделии'] / 100)), ids['Цена Розн., за шт'] * 0.6 * 0.8 * (1 - (ids['Скидка на изделии'] / 100))
    )
)

In [50]:
ids['Наценка'] = ids['price_personal'] / ids['Цена Закупки за 1 шт. (упр сс с НДС за единицу)'] - 1

In [52]:
ids.columns

Index(['Код склада', 'Название склада', 'Название товара',
       'Артикул поставщика', 'Проба', 'Чистый вес', 'Товарное направление',
       'Товарная группа', 'Тип изделия 1', 'Тип изделия 2', 'Тип изделия 3',
       'Группа цен', 'Скидка на изделии', 'Запрет скидки', 'Дизайн', 'Бренд',
       'Дата закупки', 'Цена Розн., за шт', 'Гендерный признак',
       'Ценовая корзина', 'Товарный кластер', 'Внутренний артикул', 'ID Сайта',
       'Цена Закупки за 1 шт. (упр сс с НДС за единицу)', 'Остаток_y',
       'Количество продаж за май', 'Ссылка на изображение', 'published',
       'price_personal', 'Наценка'],
      dtype='object')

In [53]:
ids.columns = ['Код склада', 'Название склада', 'Название товара',
    'Артикул поставщика', 'Проба', 'Чистый вес', 'Товарное направление',
    'Товарная группа', 'Тип изделия 1', 'Тип изделия 2', 'Тип изделия 3',
    'Группа цен', 'Скидка на изделии', 'Запрет скидки', 'Дизайн', 'Бренд',
    'Дата закупки', 'Цена Розн., за шт', 'Гендерный признак',
    'Ценовая корзина', 'Товарный кластер', 'Внутренний артикул', 'ID Сайта',
    'Себестоимость', 'Остаток',
    'Количество продаж за май', 'Ссылка на изображение', 'published',
    'Итоговая цена на изделии', 'Наценка']

In [55]:
stock['Дата закупки'] = pd.to_datetime(stock['Дата закупки'], dayfirst=True)

In [56]:
min_date = stock.groupby(by = 'Артикул поставщика', as_index=False)['Дата закупки'].min()

In [57]:
min_date.columns = ['article', 'date']

In [58]:
articles = df['Артикул поставщика'].tolist()

In [59]:
min_date = min_date.query('article in @articles')

In [60]:
min_date.columns = ['Артикул поставщика', 'Дата закупки']

In [62]:
ids = ids.drop(columns=['Дата закупки'])

In [63]:
ids = ids.merge(min_date, how='left', on='Артикул поставщика')

In [66]:
os.chdir(r'C:\\Users\Trenkin.Sergey\Desktop\sales')
extension = 'xlsx'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
all_filenames

['2024-03-10_2024-03-19.xlsx',
 '2024-03-20_2024-03-31.xlsx',
 '2024-04-01_2024-04-09.xlsx',
 '2024-04-10_2024-04-19.xlsx',
 '2024-04-20_2024-04-30.xlsx',
 '2024-05-01_2024-05-09.xlsx',
 '2024-05-10_2024-05-19.xlsx',
 '2024-06-01_2024-06-09.xlsx',
 '2024-06-10_2024-06-10.xlsx',
 '2024-06-10_2024-06-12.xlsx']

In [67]:
sale = pd.DataFrame()
for name in all_filenames:
    sales = pd.read_excel(name)
    sale = pd.concat([sales, sale], ignore_index= True)

In [70]:
sale['Дата'] = pd.to_datetime(sale['Дата'], dayfirst=True, format='mixed')

In [71]:
sale = sale[(sale['Дата'] >= pd.to_datetime('2024-03-14', dayfirst=True, format='mixed')) & 
            (sale['Дата'] <= pd.to_datetime('2024-06-11', dayfirst=True, format='mixed'))]

In [72]:
sale.columns = ['КБК', 'Дата', 'Продавец', 'Номер чека', 'ID продажи', 'Тов',
    'article', 'Количество', 'Номенклатура', 'Номер карты',
    'Товарная группа', 'Товарное направление', 'Тип 1',
    'Списание СберСпасибо', 'Группа наценки', 'Вес', 'Проба', 'Размер',
    'ШК', 'Цена без скидки', 'Базовая скидка', 'Скидка в руб.', 'Название',
    'Цена после скидки', 'Phone', 'Себестоимость', 'Сумма', 'Услуга',
    'Общая сумма']

In [73]:
sale = sale.query('article in @articles')

In [74]:
sale = sale.groupby(by = 'article', as_index=False)[['Количество', 'Общая сумма']].sum()

In [75]:
sale.columns = ['Артикул поставщика', 'Количество продаж', 'Общая сумма продаж']

In [76]:
ids = ids.merge(sale, how='left', on='Артикул поставщика')

In [78]:
ids.columns

Index(['Код склада', 'Название склада', 'Название товара',
       'Артикул поставщика', 'Проба', 'Чистый вес', 'Товарное направление',
       'Товарная группа', 'Тип изделия 1', 'Тип изделия 2', 'Тип изделия 3',
       'Группа цен', 'Скидка на изделии', 'Запрет скидки', 'Дизайн', 'Бренд',
       'Цена Розн., за шт', 'Гендерный признак', 'Ценовая корзина',
       'Товарный кластер', 'Внутренний артикул', 'ID Сайта', 'Себестоимость',
       'Остаток', 'Количество продаж за май', 'Ссылка на изображение',
       'published', 'Итоговая цена на изделии', 'Наценка', 'Дата закупки',
       'Количество продаж', 'Общая сумма продаж'],
      dtype='object')

In [79]:
ids.columns = ['Код склада', 'Название склада', 'Название товара',
    'Артикул поставщика', 'Проба', 'Чистый вес', 'Товарное направление',
    'Товарная группа', 'Тип изделия 1', 'Тип изделия 2', 'Тип изделия 3',
    'Группа цен', 'Скидка на изделии', 'Запрет скидки', 'Дизайн', 'Бренд',
    'Цена Розн., за шт', 'Гендерный признак', 'Ценовая корзина',
    'Товарный кластер', 'Внутренний артикул', 'ID Сайта', 'Себестоимость',
    'Остаток', 'Количество продаж за май', 'Ссылка на изображение',
    'published', 'Итоговая цена на изделии', 'Наценка', 'Минимальная дата закупки', 'Количество продаж 14.03-11.06', 'Общая сумма продаж 14.03-11.06']

In [80]:
stock

,Код склада,Название склада,Название товара,Артикул поставщика,Остаток,Проба,Чистый вес,Товарное направление,Товарная группа,Тип изделия 1,Тип изделия 2,Тип изделия 3,Группа цен,Скидка на изделии,Запрет скидки,Дизайн,Бренд,Дата закупки,"Цена Розн., за шт",Гендерный признак,Ценовая корзина,Товарный кластер,Внутренний артикул,ID Сайта,Цена Закупки за 1 шт. (упр сс с НДС за единицу)
0,4616,"Мытищи, Мира, 51, ТРЦ «ИЮНЬ»",Браслет Веревка пуст,431-00-0060-30333,1.00,585,2.09,ЦБ,ЦБ БРАСЛЕТЫ,БРАСЛЕТ,БЕЗ КАМНЕЙ,КРАСНЫЙ,ЦБ_ЮИТ_ПУСТОТЕЛЫЕ_КОРДА <3 ГР,25.00,Нет,ВЕРЕВКА ПУСТ,Значение по умолчанию,2023-02-15,34970.00,женские,ДЕШЕВАЯ,7645_ЦБ,431-00-0060-30333_ЦБ,3778045,4782.59
1,4616,"Мытищи, Мира, 51, ТРЦ «ИЮНЬ»",Браслет Веревка пуст,431-00-0060-30333,1.00,585,2.16,ЦБ,ЦБ БРАСЛЕТЫ,БРАСЛЕТ,БЕЗ КАМНЕЙ,КРАСНЫЙ,ЦБ_ЮИТ_ПУСТОТЕЛЫЕ_КОРДА <3 ГР,25.00,Нет,ВЕРЕВКА ПУСТ,Значение по умолчанию,2023-12-07,41904.00,женские,ДЕШЕВАЯ,7645_ЦБ,431-00-0060-30333_ЦБ,3778045,7976.13
2,4616,"Мытищи, Мира, 51, ТРЦ «ИЮНЬ»",Браслет Веревка пуст,431-00-0060-30333,1.00,585,2.18,ЦБ,ЦБ БРАСЛЕТЫ,БРАСЛЕТ,БЕЗ КАМНЕЙ,КРАСНЫЙ,ЦБ_ЮИТ_ПУСТОТЕЛЫЕ_КОРДА <3 ГР,25.00,Нет,ВЕРЕВКА ПУСТ,Значение по умолчанию,2023-12-07,42292.00,женские,ДЕШЕВАЯ,7645_ЦБ,431-00-0060-30333_ЦБ,3778045,8042.10
3,4616,"Мытищи, Мира, 51, ТРЦ «ИЮНЬ»",Браслет Веревка пуст,431-00-0060-30333,1.00,585,2.21,ЦБ,ЦБ БРАСЛЕТЫ,БРАСЛЕТ,БЕЗ КАМНЕЙ,КРАСНЫЙ,ЦБ_ЮИТ_ПУСТОТЕЛЫЕ_КОРДА <3 ГР,25.00,Нет,ВЕРЕВКА ПУСТ,Значение по умолчанию,2023-01-26,36593.00,женские,ДЕШЕВАЯ,7645_ЦБ,431-00-0060-30333_ЦБ,3778045,5132.15
4,4616,"Мытищи, Мира, 51, ТРЦ «ИЮНЬ»",Браслет Ромб двойной пуст с а/г,411-01-0035-30176,1.00,585,0.65,ЦБ,ЦБ БРАСЛЕТЫ,БРАСЛЕТ,БЕЗ КАМНЕЙ,КРАСНЫЙ,"ЦБ_ПУСТОТЕЛЫЕ <=1,5 ГРАММ",30.00,Нет,РОМБ ДВОЙНОЙ ПУСТ,ADRIATICA,2023-04-12,12513.00,женские,ДЕШЕВАЯ,7730_ЦБ,411-01-0035-30176_ЦБ,1674395,1436.74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6491294,3963,"Ярославль, пос.Нагорный, Дорожная, 6а, ТРК «Яр...",Подвес Крест черн.,ПК-10104,1.00,925,4.53,СИ,СИ ПОДВЕС БК,ПОДВЕС КУЛЬТОВЫЙ,БЕЗ КАМНЕЙ,ЧЁРНЫЙ,СИ_ПОДВЕС_КУЛЬТ_СРЕДН,40.00,Нет,СИ КРЕСТ,Значение по умолчанию,2024-05-17,4148.00,унисекс,ДОРОГАЯ,6423_СИ,ПК-10104_СИ,1892970,883.35
6491295,3963,"Ярославль, пос.Нагорный, Дорожная, 6а, ТРК «Яр...",Подвес Крест черн.,ПК-064,1.00,925,3.10,СИ,СИ ПОДВЕС БК,ПОДВЕС КУЛЬТОВЫЙ,БЕЗ КАМНЕЙ,ЧЁРНЫЙ,СИ_ПОДВЕС_КУЛЬТ_СРЕДН,40.00,Нет,СИ КРЕСТ,Значение по умолчанию,2024-05-17,2982.00,унисекс,ДОРОГАЯ,6423_СИ,ПК-064_СИ,1672265,604.49
6491296,3963,"Ярославль, пос.Нагорный, Дорожная, 6а, ТРК «Яр...",Подвес Крест черн.,ПК-909,1.00,925,4.64,СИ,СИ ПОДВЕС БК,ПОДВЕС КУЛЬТОВЫЙ,БЕЗ КАМНЕЙ,ЧЁРНЫЙ,СИ_ПОДВЕС_КУЛЬТ_СРЕДН,40.00,Нет,СИ КРЕСТ,Значение по умолчанию,2024-05-15,4815.00,унисекс,ДОРОГАЯ,6423_СИ,ПК-909_СИ,1838978,904.80
6491297,3963,"Ярославль, пос.Нагорный, Дорожная, 6а, ТРК «Яр...",Серьги-пусеты с янтарем,912040357ba,1.00,585,1.60,ПДК,ПДК СЕРЬГИ,СЕРЬГИ,С ПОЛУДРАГОЦЕННЫМИ КАМНЯМИ,КРАСНЫЙ,ШОК_ЦЕНА,0.00,Да,ДК С ОДНИМ КАМНЕМ,Значение по умолчанию,2023-06-11,8910.00,женские,ДЕШЕВАЯ,ДК_2564,912040357ba_ПДК,1581826,4351.82


In [81]:
center = stock[stock['Код склада'] == '3075']
center = center.groupby(by = 'Артикул поставщика', as_index=False)['Остаток'].sum()
center.columns = ['Артикул поставщика', 'Остаток на центральном складе']

In [82]:
kolambus = stock[stock['Код склада'] == '5442']
kolambus = kolambus.groupby(by = 'Артикул поставщика', as_index=False)['Остаток'].sum()
kolambus.columns = ['Артикул поставщика', 'Остаток на складе коламбуса']

In [83]:
federal = pd.read_excel(r'C:\Users\Trenkin.Sergey\Desktop\Федеральные склады 11.06.xlsx')

In [84]:
federal = federal['outlet_name'].tolist() 

In [89]:
stock.columns = ['Код склада', 'name', 'Название товара',
    'Артикул поставщика', 'Остаток', 'Проба', 'Чистый вес',
    'Товарное направление', 'Товарная группа', 'Тип изделия 1',
    'Тип изделия 2', 'Тип изделия 3', 'Группа цен', 'Скидка на изделии',
    'Запрет скидки', 'Дизайн', 'Бренд', 'Дата закупки', 'Цена Розн., за шт',
    'Гендерный признак', 'Ценовая корзина', 'Товарный кластер',
    'Внутренний артикул', 'ID Сайта',
    'Цена Закупки за 1 шт. (упр сс с НДС за единицу)']

In [90]:
federal = stock.query('name in @federal')

In [94]:
federal = federal.groupby(by = 'Артикул поставщика', as_index=False)['Остаток'].sum()
federal.columns = ['Артикул поставщика', 'Остаток на федеральных складах']

In [95]:
federal

,Артикул поставщика,Остаток на федеральных складах
0,К-392,1.00
1,0-017,5.00
2,00-03-0858,1.00
3,00008-П,1.00
4,00013Ч,1.00
...,...,...
19609,с35-04,6.00
19610,с462-20,2.00
19611,с9-16,4.00
19612,сCRM-026#КЕРАМИК,1.00


In [96]:
ids = ids.merge(federal, how='left', on='Артикул поставщика')

In [97]:
ids = ids.merge(kolambus, how='left', on='Артикул поставщика')

In [98]:
ids = ids.merge(center, how='left', on='Артикул поставщика')

In [99]:
ids.to_excel('предварительный.xlsx', index=False)

In [101]:
ids['Сумма в себестоимости на остатках'] = ids['Себестоимость'] * ids['Остаток']

In [103]:
zapret = pd.read_excel(r'C:\Users\Trenkin.Sergey\Desktop\запрет Артикулы к размещению.xlsx')

In [104]:
zapret.columns = ['ТГ', 'article', 'Описание', 'Рейтинг по размещению 30.04.2024',
    'Артикул сайта', 'Опубликован', 'Остаток на 30.04.24',
    'Себестоимость остатка на 30.04.2024',
    'Среднее количество продаж, шт/мес', 'Средняя сумма продаж, руб/мес',
    'Cредняя себестоимость, руб/мес', 'Средняя валовая прибыль,  руб/мес',
    'Средняя цена продажи, шт', 'Причина блокировки']

In [105]:
zapret = zapret['article'].tolist()

In [108]:
ids['Запрет к размещению'] = np.nan
for i in ids.index:
    if ids['Артикул поставщика'][i] in zapret:
        ids['Запрет к размещению'][i] = True
    else:
        ids['Запрет к размещению'][i] = False

In [109]:
ids

,Код склада,Название склада,Название товара,Артикул поставщика,Проба,Чистый вес,Товарное направление,Товарная группа,Тип изделия 1,Тип изделия 2,Тип изделия 3,Группа цен,Скидка на изделии,Запрет скидки,Дизайн,Бренд,"Цена Розн., за шт",Гендерный признак,Ценовая корзина,Товарный кластер,Внутренний артикул,ID Сайта,Себестоимость,Остаток,Количество продаж за май,Ссылка на изображение,published,Итоговая цена на изделии,Наценка,Минимальная дата закупки,Количество продаж 14.03-11.06,Общая сумма продаж 14.03-11.06,Остаток на федеральных складах,Остаток на складе коламбуса,Остаток на центральном складе,Сумма в себестоимости на остатках,Запрет к размещению
0,4616,"Мытищи, Мира, 51, ТРЦ «ИЮНЬ»",Браслет Веревка пуст,431-00-0060-30333,585,2.09,ЦБ,ЦБ БРАСЛЕТЫ,БРАСЛЕТ,БЕЗ КАМНЕЙ,КРАСНЫЙ,ЦБ_ЮИТ_ПУСТОТЕЛЫЕ_КОРДА <3 ГР,25.00,Нет,ВЕРЕВКА ПУСТ,Значение по умолчанию,34970.00,женские,ДЕШЕВАЯ,7645_ЦБ,431-00-0060-30333_ЦБ,3778045,4782.59,3501.00,140.00,https://static2.585.cloud/media/products/d5758...,True,12589.20,1.63,2022-08-10,309.00,4911964.79,23.00,37.00,83.00,16743847.59,False
1,4616,"Мытищи, Мира, 51, ТРЦ «ИЮНЬ»",Браслет Ромб двойной пуст с а/г,411-01-0035-30176,585,0.65,ЦБ,ЦБ БРАСЛЕТЫ,БРАСЛЕТ,БЕЗ КАМНЕЙ,КРАСНЫЙ,"ЦБ_ПУСТОТЕЛЫЕ <=1,5 ГРАММ",30.00,Нет,РОМБ ДВОЙНОЙ ПУСТ,ADRIATICA,12513.00,женские,ДЕШЕВАЯ,7730_ЦБ,411-01-0035-30176_ЦБ,1674395,1436.74,2635.00,147.00,https://static2.585.cloud/media/products/6bedb...,True,4204.37,1.93,2022-08-03,357.00,1967853.00,14.00,10.00,78.00,3785809.90,False
2,4616,"Мытищи, Мира, 51, ТРЦ «ИЮНЬ»","Серьги, фианит",1241689,585,1.98,ИФ,ИФ СЕРЬГИ,СЕРЬГИ,С ФИАНИТАМИ,КРАСНЫЙ,ИФ.БК_СР,34.00,Нет,ИФ С ОДНИМ КАМНЕМ,Значение по умолчанию,39382.00,женские,СРЕДНЯЯ,ИФ_1044,1241689_ИФ,2735473,6802.13,506.00,25.00,https://static2.585.cloud/media/products/ff494...,True,12476.22,0.83,2022-10-10,62.00,791420.00,4.00,NaN,15.00,3441877.78,False
3,4616,"Мытищи, Мира, 51, ТРЦ «ИЮНЬ»",серьги с бриллиантом с родир.,2436-151-00-00,585,3.06,ДК,ДК СЕРЬГИ,СЕРЬГИ,С ДРАГОЦЕННЫМИ КАМНЯМИ,КРАСНЫЙ,ДК_СРЕДНЯЯ,66.00,Нет,ДК С ОДНИМ КАМНЕМ,Значение по умолчанию,163994.00,женские,СРЕДНЯЯ,ДК_2090,2436-151-00-00_ДК,4058155,10755.11,334.00,7.00,https://static2.585.cloud/media/products/501d7...,True,26763.82,1.49,2022-07-02,17.00,464700.00,3.00,2.00,2.00,3592206.74,False
4,4616,"Мытищи, Мира, 51, ТРЦ «ИЮНЬ»",Подвес род.,2035076/9,925,0.50,СИ,СИ ПОДВЕС БК,ПОДВЕС ДЕКОРАТИВНЫЙ,БЕЗ КАМНЕЙ,БЕЛЫЙ С РОДИРОВАНИЕМ,СИ_ПОДВЕС_ДЕКОР_СРЕДН,40.00,Нет,СИ ПРОЧЕЕ,Значение по умолчанию,1150.00,женские,ЛЕГКОВЕС,6400_СИ,2035076/9_СИ,3558970,195.00,1088.00,131.00,https://static2.585.cloud/media/products/21c90...,True,331.20,0.70,2022-07-12,288.00,129163.00,6.00,4.00,28.00,212160.00,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17358,3075,Центральный склад Регент Голд,"Кольцо с бриллиантами выращ., с родир.",20-12-Ф000-13341,585,1.09,ДК,ДК КОЛЬЦА,КОЛЬЦО,С ИСКУССТВЕННЫМИ КАМНЯМИ,БЕЛЫЙ,ДК_ДОРОГАЯ,0.00,Нет,ДК МНОГОКАМЕНКА,Значение по умолчанию,NaN,женские,ДОРОГАЯ,ДК_2027,20-12-Ф000-13341 _ДК,,24924.21,95.00,NaN,NaN,NaN,NaN,NaN,2024-06-10,NaN,NaN,NaN,NaN,95.00,2367799.95,False
17359,3075,Центральный склад Регент Голд,Подвес икона Семистрельная,52-0Z-0000-13522,585,1.19,БК,БК ПОДВЕС КУЛЬТ,ПОДВЕС КУЛЬТОВЫЙ,БЕЗ КАМНЕЙ,КРАСНЫЙ,ИФ.БК_ДЕШ,0.00,Нет,ИФ ИКОНА,Значение по умолчанию,NaN,унисекс,ДЕШЕВАЯ,БК_1568,52-0Z-0000-13522_БК,,4197.12,50.00,NaN,NaN,NaN,NaN,NaN,2024-06-08,NaN,NaN,NaN,NaN,50.00,209856.00,False
17360,3075,Центральный склад Регент Голд,Подвес,50-00-0000-13608#ER,585,1.09,БК,БК ПОДВЕС ДЕКОР,ПОДВЕС ДЕКОРАТИВНЫЙ,БЕЗ КАМНЕЙ,КРАСНЫЙ,ИФ.БК_СР,0.00,Нет,ИФ ПРОЧЕЕ,Значение по умолчанию,NaN,Унисекс,СРЕДНЯЯ,БК_1593,50-00-0000-13608#ER_БК,,3980.90,55.00,NaN,NaN,NaN,NaN,NaN,2024-06-08,NaN,NaN,NaN,NaN,55.00,218949.50,False
17361,3075,Центральный склад Регент Голд,"Подвес позолота, культивированный жемчуг",18.104,925,2.30,ВЫРИЦА,ВЫРИЦА СЕРЕБРО,ПОДВЕС,С ЖЕМЧУГОМ,ЖЕЛТЫЙ,СИ_ВЫРИЦА_

In [175]:
df['Запрет к размещению'].value_counts()

Запрет к размещению
False    17311
True        36
Name: count, dtype: int64

In [111]:
ids['rank1'] = ids['Остаток на федеральных складах'].rank()
ids['rank2'] = ids['Сумма в себестоимости на остатках'].rank()
ids['rank3'] = ids['Общая сумма продаж 14.03-11.06'].rank()

In [112]:
ids['rank'] = (ids['rank1'] + ids['rank2'] + ids['rank3'])/3

In [114]:
ids = ids.sort_values(by = ['rank'], ascending=False)

In [117]:
ids = ids.drop(columns=['index'])

In [115]:
ids = ids.reset_index()
ids['Рейтинг'] = np.nan
for i in ids.index:
    ids['Рейтинг'][i] = i + 1

In [118]:
ids = ids.drop(columns=['rank1', 'rank2', 'rank3'])

In [240]:
df = df.drop(columns=['Скидка на изделии', 'level_0', 'index'])

In [119]:
ids.to_excel('final.xlsx', index=False)

In [225]:
df = pd.read_excel('final.xlsx')

In [122]:
ids.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17363 entries, 0 to 17362
Data columns (total 39 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   Код склада                         17363 non-null  object        
 1   Название склада                    17363 non-null  object        
 2   Название товара                    17363 non-null  object        
 3   Артикул поставщика                 17363 non-null  object        
 4   Проба                              17363 non-null  object        
 5   Чистый вес                         17363 non-null  float64       
 6   Товарное направление               17363 non-null  object        
 7   Товарная группа                    17363 non-null  object        
 8   Тип изделия 1                      17363 non-null  object        
 9   Тип изделия 2                      17363 non-null  object        
 10  Тип изделия 3                     